In [ ]:
import cv2
import numpy as np
import glob

class ImageProcessor():
    # Initializes standard image sizes: width and height = 64. Changing these values changes values in resize functions
    def __init__(self, image_size=(64, 64)):
        self.target_size = image_size

    # open and read images from specified file path
    def load_image(self, image_path):
        # Loads an image from the given path and returns it as a numpy array
        image = cv2.imread(image_path)
        if image is None:
            print(f"ERROR: Unable to read image at {image_path}")
        return image

     # ensures that the input is not an empty array
    def check_if_valid(self, image):
        if not isinstance(image, np.ndarray):
            return False  # return False if not valid
        return True  # return True if valid

     # checks to see if image is grayscale
    def convert_to_grayscale(self, image):
        # RGB colored images usually have 3 dimensions (width, height, color)
        # if the thrid dimesion of the tuple (represented by index of 2) has the color channel of RGB, then convert to grayscale
        if len(image.shape) == 3 and image.shape[2] == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return image

    #scales pixels to [0,1]
    def normalize_image(self, image):
      #converts to float32 in case data is integers
      normalized_image = image.astype(np.float32) / 255.0
      return normalized_image

    # used to sharpen images using cv2 kernels (if data requires image sharpening)
    def sharpen_image (self, image):
      kerenel_sharpening = np.array([[-1,-1,-1],
                                      [-1,9,-1],
                                      [-1,-1,-1]])
      sharpened_image = cv2.filter2D(image, -1, kerenel_sharpening)
      return sharpened_image

    # (1) resizes the image to target size using INTER_AREA interpolation, recommended when downscaling original image
    def downscale_image(self, image):
      return cv2.resize(image, self.target_size, interpolation=cv2.INTER_AREA)

    # (2) resizes the image to target size using INTER_CUBIC interpolation, recommended when upscaling original image
    def upscale_image(self, image):
      return cv2.resize(image, self.target_size, interpolation=cv2.INTER_CUBIC)

    # (3) resizes an image to target size while maintaining ratios (uses black padding to cover open space), prevents distortion.
    # might need to change this function a little if image is not grayscaled
    def resize_with_aspect_ratio(self, image, pad_color):
      h, w = image.shape[:2]  #gets original dimensions of image
      target_h, target_w = self.target_size  #gets target dimensions

      aspect_original = w / h  #original aspect ratio
      aspect_target = target_w / target_h  #target aspect ratio

      # If the image already fits within the target size, no padding is required.
      if h == target_h and w == target_w:
          return image  # Return the image without changes if no padding is needed

      # if our model does not need all images to be exact same size, padding not necessary
      padded_image = np.zeros((target_h, target_w), dtype=np.uint8)  #creates blank image with target size (padding)
      padded_image[:] = pad_color  #sets padded image color to black

      #if original image is wider, recalculates height and sets width to max width (target width)
      if aspect_original > aspect_target:
          new_w = target_w
          new_h = int(new_w / aspect_original)
          #resizes image and uses either INTER_AREA or INTER_CUBIC depending on downscale or upscale
          if new_h <= h:
            resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
          else:
            resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)

          y_offset = (target_h - new_h) // 2  #calculates offset to center image on padded image (idk if we need this tbh)
          padded_image[y_offset:y_offset + new_h, :] = resized  #places image on padded image

      #if original image is taller, recalculates width and sets height to max height (target height)
      else:
          new_h = target_h
          new_w = int(new_h * aspect_original)
          if new_w <= w:
            resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
          else:
            resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_CUBIC)

          x_offset = (target_w - new_w) // 2
          padded_image[:, x_offset:x_offset + new_w] = resized

      return padded_image  #returns original image

    #cleans landmark data by handling missing values and outliers
    #assumes all images have the same number of landmarks, so if not, we will have to change this function a bit
    #if all images have the same number of landmarks, we still need to change a little to account for images that might be missing landmarks (NaN)
    def clean_landmarks(self, landmarks_data):
      if landmarks_data:
        num_landmarks = len(landmarks_data[0]) #accesses first image and returns number of landmarks, assumes all images have the same # of landmarks
      else:
        return []  #functions stops here if landmark data is empty

      #creates new list of landmark data by iterating through elements in original data
      cleaned_landmarks = [list(landmark) for landmark in landmarks_data]

      #replaces missing (None) values with mean
      for i in range (num_landmarks):
        values = [landmark[i] for landmark in cleaned_landmarks if landmark is not None]  #gets all valid landmark data
        mean_val = np.mean(values)  #calculates mean
        for j in range(len(cleaned_landmarks)): # Iterates through each landmark set in cleaned_landmarks
            if cleaned_landmarks[j][i] is None:
              cleaned_landmarks[j][i] = mean_val # If None, replace with the calculated mean

      #Outlier remover
      for i in range(num_landmarks):
        values = [landmark[i] for landmark in cleaned_landmarks if landmark is not None]
        if not values:
          print("Warning: No valid data to calculate quartiles for a landmark. Returning unfiltered landmarks.")
          return cleaned_landmarks  # Returns the landmarks without filtering if there is no data to calculate quartiles
        q1 = np.quantile(values, 0.25)  #calculates first quartile
        q3 = np.quantile(values, 0.75)  #calculates third quartile
        iqr = q3 - q1  #interquartile range
        lower_bound = q1 - 1.5 * iqr  #any value below this is considered potential outlier
        upper_bound = q3 + 1.5 * iqr  #any value above this is considered potential outlier

        cleaned_landmarks_filtered = []  #create a new list to store cleaned landmarks
        for landmark in cleaned_landmarks:  # iterate through each landmark
          is_outlier = False
          for j in range(num_landmarks):  #iterate through each coordinate of the landmark
            if not (lower_bound <= landmark[j] <= upper_bound):  #check if the coordinate is outside the bounds
              is_outlier = True  #if any coordinate is an outlier, mark the landmark as an outlier
              break  #no need to check other coordinates for this landmark

          #if the landmark is not an outlier, it is added to the filtered list
          if not is_outlier:
            cleaned_landmarks_filtered.append(landmark)  #add it to the filtered list

      return cleaned_landmarks_filtered #return the filtered landmark data list at the end of the function

      def clean_multiple_landmarks(self, landmarks_list):
          cleaned_data = []
          for landmarks_data in landmarks_list:
              cleaned_data.append(self.clean_landmarks(landmarks_data))
          return cleaned_data

      #function to process a single image
      def process_image(image_path, processor, landmarks_data=None):
          image = processor.load_image(image_path)
          if image is None:
              return None, image_path #if image not loaded correctly, returns none along with image path to make error handling easier

          #checks if image is a valid numpy array
          if not processor.check_if_valid(image):
              print(f"Error: Image is not a numpy array: {image_path}")
              return None, image_path #if image not a numpy array, returns none along with image path

          image = processor.convert_to_grayscale(image)
          image = processor.normalize_image(image)
          image = processor.resize_with_aspect_ratio(image)

          if landmarks_data is not None: # Check if landmarks are provided
              cleaned_landmarks = clean_landmarks(landmarks_data)
              return image, image_path, cleaned_landmarks #Return cleaned landmarks
          else:
              return image, image_path